# imports

In [0]:
import numpy as np
import scipy.io
import pandas as pd
from random import randint
from time import time
sin=np.sin
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
asin=np.arcsin
from numpy.linalg import inv
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
global c_i
c_i=2**8

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
######optional to save all features in a seperate folder
import os

##### choose ur own folder to save them

os.chdir('/content/drive/My Drive/final project /data-channels')

#Custom function

In [0]:
def randperm(n):
    r_perm=[]
    while len(r_perm)!=n:
        k=randint(1,n)
        if k not in r_perm:
            r_perm.append(k)
        else:
            pass
    return r_perm

In [0]:
def noofclass(fea,label=True):
    n_class=0
    labels=[]
    for i in range(fea[0].shape[0]):
        labels.append(fea[0][i])
        if fea[0][i]>n_class:
            n_class=fea[0][i]
        else:
            n_class=n_class
    if label==False:
        return int(n_class+1)
    else:
        labels=np.matrix(labels)
        return labels.transpose()

In [0]:
def linear_kernal(a,b):
  return np.matmul(a,b.transpose())

In [0]:
def normalize(a,b):
  temp=a+b
  min=np.min(temp)
  max=np.max(temp)
  #print(min,max)
  u=2*((temp-min)/(max-min))
  u=u-1
  return u 

In [0]:
def sigmoid_inv(a):
  temp=1/a
  temp=temp-1
  temp=np.log(temp)
  temp=-temp
  return temp

In [0]:
import math

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [0]:
def normalize_inv(a):
  return (np.max(a)-a)/(np.max(a)-np.min(a))


# Morres-pseudo inverse

In [0]:
def relu(features):
  return np.maximum(0,features)
def sigmoid(features):
  return 1.0 / (1.0 + np.exp(-features))
def linear(features,weights,bias):
   return np.dot(features, np.transpose(weights)) + bias

In [0]:
### function for morres-pseudo inverse 
### Input: features, target, weights, bias
### Output: Morres-matrix, Beta (output weights)
def pseudoinverse(features, targets,weights,bias,forgettingFactor=0.999):
    (numSamples, numOutputs) = targets.shape
    assert features.shape[0] == targets.shape[0]

    H = linear(features,weights,bias)

    H = sigmoid(H)
    
    Ht = np.transpose(H)


    M = (1/forgettingFactor) * M - np.dot((1/forgettingFactor) * M,
                                     np.dot(Ht, np.dot(
                                       pinv(np.eye(numSamples) + np.dot(H, np.dot((1/forgettingFactor) * M, Ht))),
                                       np.dot(H, (1/forgettingFactor) * M))))
    beta = beta + np.dot(M, np.dot(Ht, targets - np.dot(H,beta)))

    return (M,beta)


#sample_g


In [0]:
def sample_g(fea):
    nclass=noofclass(fea,False)
    train_per_image=100;
    fdatabase_label=noofclass(fea,True)
    tr_idx=[]
    ts_idx=[]
    tr_list=[]
    ts_list=[]
    idx_label=0
    for i in range(nclass):
        idx_label=[]
        for j in range(len(fdatabase_label)):
            a=int(fdatabase_label[j])
            if a==i:
                idx_label.append(j)
        num=len(idx_label)
        idx_rand=randperm(num-1)
        tr_num =train_per_image
        for k in range(tr_num):
            temp=idx_label[idx_rand[k]]
            tr_list.append(temp)
        for l in range(tr_num-1,num-1):
            temp=idx_label[idx_rand[l]]
            ts_list.append(temp)
        
    return np.array(tr_list),np.array(ts_list)

#First General Layer - 
  Subspace Feature Extraction

In [0]:
def step1(training,testing):
  X=training
  y=testing
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
  print('traindata : ',X_train.shape,'testdata : ',X_test.shape,'train_label : ',y_train.shape,'test_label : ',y_test.shape)
  return X_train, X_test, y_train, y_test

In [0]:
#weights and bias generation on random
#for the first layer

def step2(data_set):  
  dim=data_set.shape
  weights=np.random.randn(1,dim[0])
  bias=np.random.randn(1,1)
  H=np.dot(weights,data_set)+bias
  #print('weights : ',weights.shape,'data : ',data_set.shape,'bias : ',bias.shape,'H size : ',H.shape)
  return H

In [0]:
def step3(subspacefeature,labels):
  temp=inv(c_i+np.dot(subspacefeature,subspacefeature.transpose()))
  #print(labels.shape,temp.shape)
  temp=np.dot(subspacefeature.transpose(),temp)
  #print(temp.transpose().shape)
  y=labels
  weights=(y*temp).transpose()
  #print(weights.shape,'-weights')
  temp=weights*subspacefeature
  ind = np.ones((1,subspacefeature.shape[1]))
  y=np.matrix(y).transpose()
  y=y*ind
  #print(temp.shape,y.shape,'-v')
  bias=mean_squared_error(temp,y)
  bias=np.sqrt(bias)
  return weights,bias

In [0]:
def step4(weights,bias,subspacefeature,labels):
  ind = np.ones((1,subspacefeature.shape[1]))
  y=labels
  y=np.matrix(y).transpose()
  y=y*ind
  e=y-(weights*subspacefeature+bias)
  return e

In [0]:
def step5(error,weights,subspacefeature):#step 5  &6
  #c_i=np.random.randint(1,10)
  temp=c_i+np.dot(weights,weights.transpose())
  temp=np.linalg.pinv(temp)
  temp=np.dot(weights.transpose(),temp.transpose())
  temp=np.dot(weights.transpose(),temp.transpose())
  p=error*temp
  p=normalize(p,subspacefeature)
  #print(p.shape)
  return p

In [0]:
### APPLY BOOSTING PHASE HERE
def step7(weight_fl,bias_fl,input_data,normalized_error):
  #c_i=np.random.randint(1,10)
  a=weight_fl
  b=bias_fl
  x=input_data
  p=normalized_error
  temp=np.ones(shape=(p.shape[1],x.shape[0]))
  #print(temp.shape)
  p_temp=p
  p=p*temp
  #print(p.shape)
  a_temp=(p*np.dot(x.transpose(),np.linalg.pinv(c_i+np.dot(x,x.transpose()))).transpose())
  a_f=a+0.5*(a_temp-a)
  temp=np.array(a_f)*x
  #print(p.shape)
  b_f=np.sqrt(mean_squared_error(temp,p_temp))
  #print(type(a_f),type(x),type(p))
  h_f=np.array(a_f)*x+b_f
  return h_f,a_f,b_f

#Second Generel Layer -
  Subspace features combination

In [0]:
#def fea_comb(subnetwork_nodes,data_channels):
def fea_comb(feature_array,subnetowk_nodes,data_channels):
  for i in range(1,data_channels+1):
    for j in range(1,subnetwork_nodes):
      #temp="ch%s_fea_%s"%(i,j)
      temp=feature_array
      if i==1 and j==1:
        #h=np.loadtxt(temp)
        h=temp
      else:
        prev_h=h
        #h=np.loadtxt(temp)
        h=feature_array
        final_h=linear_kernal(h.transpose(),prev_h.transpose())
  return final_h

# Third General Layer - 
  Classifier with Subnetowrk Nodes

In [0]:
def last_layer(final_features,test_labels):
  c2=2**12
  C=3
  N=10
  #step 1
  h=final_features
  for c in range(C):
    if c==0:
      prev_e=test_labels
    else:
      weights=c2+np.dot(h,h.transpose())
      print(prev_e.shape)
      temp=np.ones(prev_e.shape)
      print(temp.shape)
      
      temp=prev_e+temp
      print(temp.shape)
      
      temp=sigmoid_inv(temp)
      print(temp.shape)
      temp=np.dot(temp,np.dot(h.transpose(),np.linalg.pinv(weights)))
      #weights=sigmoid_inv(prev_e+np.ones(prev_e.shape))*np.dot(h.transpose(),np.linalg.pinv(weights))
      bias=np.sum(np.matmul(weights.transpose(),h.transpose())-(sigmoid_inv(prev_e))   )   
      bias=bias/N
      g=normalize_inv(sigmoid(weights*h.transpose()+bias))
      in_pro=np.inner(prev_e,g.transpose()).transpose()
      near_int=np.around(g)**2
      output_weights=in_pro/near_int
      e_c=prev_e-np.dot(output_weights,sigmoid(weights*ouput_weights+bias))
  return e_c
  #step 2

⟨,⟩ - inner product
|| || - nearest integer

#main

In [44]:
mat = scipy.io.loadmat('/content/drive/My Drive/final project /Matlabcode-feature-matrix/spatialpyramidfeatures4scene15.mat')
featureMat=mat['featureMat']
labelMat=mat['labelMat']
labelMat=np.argwhere(labelMat==1)
temp=[]
b=[]
for i in range(len(labelMat)):
  value=labelMat[i][0]
  b.append(labelMat[i][1])
  temp.append(value)
featureMat=featureMat.transpose()
labelMat=np.array(temp)
fea=np.column_stack([labelMat,featureMat])
fea=fea.transpose()
print("label Shape: ",labelMat.shape,'\nfeature Shape: ',featureMat.shape)
print('fea Shape: ',fea.shape)
train_idx,test_idx=sample_g(fea)
tr_fea=fea[: , train_idx].transpose()
ts_fea=fea[:,  test_idx].transpose()
print('train fea: ',tr_fea.shape,'test fea: ',ts_fea.shape)
train_data=tr_fea
test_data=ts_fea
train_labels=labelMat[train_idx]
test_labels=labelMat[test_idx]
print('train labels: ',train_labels.shape,'test_labels: ',test_labels.shape)


label Shape:  (4485,) 
feature Shape:  (4485, 3000)
fea Shape:  (3001, 4485)
train fea:  (1500, 3001) test fea:  (2985, 3001)
train labels:  (1500,) test_labels:  (2985,)


 matrix shape - (rows,columns)
 



In [0]:
train_inputs,test_inputs,train_labels,test_labels=tr_fea,ts_fea,train_labels,test_labels #step1

data_channels=10
subnetwork_nodes=3
##First layer call
for i in range(1,data_channels+1):
  for j in range(1,subnetwork_nodes+1):
      start_time=time()
      sub=step2(train_inputs) #step2
      w,b=step3(sub,train_labels) #step3
      error=step4(w,b,sub,train_labels) #step 4
      norm_err=step5(error,w,sub) #step 5 and 6
      h_f,a_f,b_f=step7(w,b,train_inputs,norm_err) #step7
      ###
      ### AFTER BOOSTING PHASE IS DONE FOR THE FIRST TIME 
      ### SAVE THE WEIGHTS IN A FILE
      ### 
      ### AFTER THE FIRST BLOCK OF DATA IS SENT THROUGH WE WILL USE THE PREVIOUSLY 
      ### SAVED WEIGHTS FILE AND ADD ON TO IT AND SAVE IT
      ###
      ### THAT COMPLETES THE SEQUENTIAL LEARNING PHASE
      ###
      end_time=time()
      h=fea_comb(h_f,subnetwork_nodes,data_channels)
train_time=end_time-start_time

print('train time : ',train_time)

# Print statement


In [0]:
print("------------------------------------------------------------------")
print('***train data*** : ')
print("------------------------------------------------------------------")

print(train_inputs)
print("------------------------------------------------------------------")
print('***sub*** : ')
print("------------------------------------------------------------------")

print(sub)
print("------------------------------------------------------------------")
print('***error*** : ')
print("------------------------------------------------------------------")
print(error)
print("------------------------------------------------------------------")
print('***norm_error*** : ')
print("------------------------------------------------------------------")
print(norm_err)
print("------------------------------------------------------------------")
print('**h_f*** : ')
print("------------------------------------------------------------------")

print(h_f)
print("------------------------------------------------------------------")
print('***h*** : ')
print("------------------------------------------------------------------")

print(h)

In [0]:
print('weights before updation\n',w)
print('bias before updation \n',b)
print('-----------------------------------------------------------------')
print("weights after updation \n",a_f)
print("bias after updation \n",b_f)

# final output

In [0]:
last_layer(h,train_labels)